##### 從「臺北市公共自行車2.0租借紀錄」- 讀取「資料資源下載網址」

In [3]:
import pandas as pd
import requests
import zipfile
from io import BytesIO
import io

In [4]:
import pandas as pd
url="https://tcgbusfs.blob.core.windows.net/dotapp/youbike_second_ticket_opendata/YouBikeHis.csv"
df = pd.read_csv(url)


#### 讀取資料發現fileurl 是zip檔

In [5]:
df.head()

,UpdateTime,SrcUpdateTime,fileinfo,fileURL
0,2024-01-16T15:50:23+08:00,2024-01-16T15:50:23+08:00,2023年11月,https://tcgbusfs.blob.core.windows.net/dotapp/...
1,2024-01-16T15:50:23+08:00,2024-01-16T15:50:23+08:00,2023年10月,https://tcgbusfs.blob.core.windows.net/dotapp/...
2,2024-01-16T15:50:23+08:00,2024-01-16T15:50:23+08:00,2023年9月,https://tcgbusfs.blob.core.windows.net/dotapp/...
3,2024-01-16T15:50:23+08:00,2024-01-16T15:50:23+08:00,2023年8月,https://tcgbusfs.blob.core.windows.net/dotapp/...
4,2023-09-15T13:55:05+08:00,2023-09-15T13:55:05+08:00,2023年7月,https://tcgbusfs.blob.core.windows.net/dotapp/...


#### Step 1: 
- 下載第一筆zip 
- 解壓縮 
- 轉df (使用pandas) 
- 命名＆存CSV

In [6]:
fileurl=df["fileURL"][0]

In [7]:
import requests
import zipfile
import io

response =requests.get(fileurl)

In [8]:
from io import BytesIO
zip_content = BytesIO(response.content)
    
# Extract the contents of the zip file（提取 zip 檔案的內容）
with zipfile.ZipFile(zip_content, 'r') as zip_ref:
    # Assuming there's only one file in the zip, extract it（假設 zip 中只有一個文件，請將其解壓縮）
    file_in_zip = zip_ref.namelist()[0]
    with zip_ref.open(file_in_zip) as csv_file:
        # Read the CSV file into a pandas DataFrame（將 CSV 檔案讀入 pandas DataFrame）
        transform_df = pd.read_csv(csv_file,header=None)

In [9]:
transform_df.to_csv("bike_usage_history.csv")

#### Step 2: 
- 逐筆
    - 下載zip & 解壓縮
    - 轉df
    - 定義csv名稱（給予規則命名）

In [ ]:
import pandas as pd
import requests
import zipfile
from io import BytesIO


#下載zip & 
fileurl=df["fileURL"][0] ##要調整
response =requests.get(fileurl)
zip_content = BytesIO(response.content)

with zipfile.ZipFile(zip_content, 'r') as zip_ref:
    file_in_zip = zip_ref.namelist()[0]
    #解壓縮
    with zip_ref.open(file_in_zip) as csv_file:
        #轉df
        transform_df = pd.read_csv(csv_file,header=None)

#存檔並定義csv名稱（給予規則命名）
transform_df.to_csv("bike_usage_history.csv") ##要調整

In [3]:
df.head()

,UpdateTime,SrcUpdateTime,fileinfo,fileURL
0,2024-01-16T15:50:23+08:00,2024-01-16T15:50:23+08:00,2023年11月,https://tcgbusfs.blob.core.windows.net/dotapp/...
1,2024-01-16T15:50:23+08:00,2024-01-16T15:50:23+08:00,2023年10月,https://tcgbusfs.blob.core.windows.net/dotapp/...
2,2024-01-16T15:50:23+08:00,2024-01-16T15:50:23+08:00,2023年9月,https://tcgbusfs.blob.core.windows.net/dotapp/...
3,2024-01-16T15:50:23+08:00,2024-01-16T15:50:23+08:00,2023年8月,https://tcgbusfs.blob.core.windows.net/dotapp/...
4,2023-09-15T13:55:05+08:00,2023-09-15T13:55:05+08:00,2023年7月,https://tcgbusfs.blob.core.windows.net/dotapp/...


In [ ]:
# 提取欄位名稱（fileinfo,fileURL）
# 將欄位fileinfo 調整顯示內容
for index, row in df.iterrows():
    fileinfo = row['fileinfo']
    fileURL = row['fileURL']
    filename = fileinfo.replace("年", "_").replace("月", "_bike_usage_history.csv")
    print(filename,fileURL)
    break

In [7]:
# Step2 串起來
for index, row in df.iterrows():
    fileinfo = row['fileinfo']
    fileURL = row['fileURL']
    filename = fileinfo.replace("年", "_").replace("月", "_bike_usage_history.csv")
    
    
    response =requests.get(fileURL)
    zip_content = BytesIO(response.content)

    with zipfile.ZipFile(zip_content, 'r') as zip_ref:
        file_in_zip = zip_ref.namelist()[0]
        #解壓縮
        with zip_ref.open(file_in_zip) as csv_file:
            #轉df
            transform_df = pd.read_csv(csv_file,header=None)

    #存檔並定義csv名稱（給予規則命名）
    transform_df.to_csv(filename) ##要調整
    
   
    print(filename,fileURL)
    # break

2023_11_bike_usage_history.csv https://tcgbusfs.blob.core.windows.net/dotapp/youbike_second_ticket_opendata/2023/2023-11/202311_YouBike2.0票證刷卡資料.zip


#### Step3:例外狀況處理 
- 3-1 總計44筆,執行Step 2 ＋將錯誤的跳過 ->挑出有問題的筆數（發現兩個錯誤）
- 3-2 處理狀況一：No columns to parse from file. Skipping to the next row...
- - #3-2-1 從zip檔案解壓縮後,只取CSV檔案
- - #3-2-2 檢查下載的CSV檔案, 發現已有欄位名稱
- 3-3 處理狀況二：'utf-8' codec can't decode byte 0xe5 & codec can't decode byte 0xe9

In [5]:
# 3-1 有錯誤就跳過
import pandas as pd
import requests
import zipfile
from io import BytesIO

for index, row in df.iterrows():
    try:
        fileinfo = row['fileinfo']
        fileURL = row['fileURL']
        filename = fileinfo.replace("年", "_").replace("月", "_bike_usage_history.csv")
        
        response = requests.get(fileURL)
        zip_content = BytesIO(response.content)

        with zipfile.ZipFile(zip_content, 'r') as zip_ref:
            file_in_zip = zip_ref.namelist()[0]
            #解壓縮
            with zip_ref.open(file_in_zip) as csv_file:
                #轉df
                transform_df = pd.read_csv(csv_file, header=None)

        #存檔並定義csv名稱（給予規則命名）
        transform_df.to_csv(filename) ##要調整

        print(filename, fileURL)
    
    except Exception as e:
        print(f"An error occurred for row {index}: {e}. Skipping to the next row...")
        continue


2023_11_bike_usage_history.csv https://tcgbusfs.blob.core.windows.net/dotapp/youbike_second_ticket_opendata/2023/2023-11/202311_YouBike2.0票證刷卡資料.zip
2023_10_bike_usage_history.csv https://tcgbusfs.blob.core.windows.net/dotapp/youbike_second_ticket_opendata/2023/2023-10/202310_YouBike2.0票證刷卡資料.zip
2023_9_bike_usage_history.csv https://tcgbusfs.blob.core.windows.net/dotapp/youbike_second_ticket_opendata/2023/2023-09/202309_YouBike2.0票證刷卡資料.zip
2023_8_bike_usage_history.csv https://tcgbusfs.blob.core.windows.net/dotapp/youbike_second_ticket_opendata/2023/2023-08/202308_YouBike2.0票證刷卡資料.zip
An error occurred for row 4: 'utf-8' codec can't decode byte 0xe5 in position 4192: invalid continuation byte. Skipping to the next row...
An error occurred for row 5: 'utf-8' codec can't decode byte 0xe5 in position 19886: invalid continuation byte. Skipping to the next row...
An error occurred for row 6: 'utf-8' codec can't decode byte 0xe9 in position 36986: invalid continuation byte. Skipping to the

In [43]:
#3-2 處理狀況一：No columns to parse from file. Skipping to the next row...
## 找其中一筆資料檢查 ????
## 此問題是zip 檔案不只一個檔案，資料夾裡放csv檔

fileurl=df["fileURL"][14]
response =requests.get(fileurl)
zip_content = BytesIO(response.content)


In [44]:
with zipfile.ZipFile(zip_content, 'r') as zip_ref:
    data = zip_ref.namelist()
    print(data)

['202209_YouBike2.0票證刷卡資料/', '202209_YouBike2.0票證刷卡資料/202209_YouBike2.0票證刷卡資料.csv']


In [14]:
#3-2-1 從zip檔案解壓縮後,只取CSV檔案
#下載zip
with zipfile.ZipFile(zip_content, 'r') as zip_ref:
    data = zip_ref.namelist()
    for file in data:
        if '.csv' in file:
            file_in_zip = file
    #解壓縮
    with zip_ref.open(file_in_zip) as csv_file:
    #     #轉df
        transform_df = pd.read_csv(csv_file,header=None)

In [22]:
#3-2-2 檢查下載的CSV檔案, 發現已有欄位名稱
## （需要處理已經有欄位名稱的就把欄位名稱改第1row,再將原本第1row資料drop）
transform_df.head()

,0,1,2,3,4,5
0,rent_time,rent_station,return_time,return_station,rent,infodate
1,2022-09-30 21:00:00,捷運大湖公園站(2號出口),2022-09-30 21:00:00,安湖三號公園,00:12:27,2022-09-30
2,2022-09-30 21:00:00,捷運大湖公園站(2號出口),2022-09-30 21:00:00,康寧公園(北側),00:12:56,2022-09-30
3,2022-09-30 21:00:00,捷運大湖公園站(2號出口),2022-09-30 21:00:00,大湖國小,00:11:10,2022-09-30
4,2022-09-30 21:00:00,捷運大湖公園站(2號出口),2022-09-30 21:00:00,康樂綠地,00:29:07,2022-09-30


In [29]:
first_cell = transform_df.iloc[0,0]  # 获取第0行数据
is_rent_time = first_cell == 'rent_time'  # 检查第0行是否等于'rent_time'
is_rent_time
if is_rent_time:  # 符合條件的把欄位名稱改第1row,再將原本第1row資料drop
    transform_df.columns = transform_df.iloc[0]
    transform_df = transform_df.drop(0)
    transform_df

True

In [34]:
## 把3-2 處理狀況一 整合進Step2 

with zipfile.ZipFile(zip_content, 'r') as zip_ref:
    data = zip_ref.namelist()
    for file in data:
        if '.csv' in file:
            file_in_zip =  file
    #解壓縮
    with zip_ref.open(file_in_zip) as csv_file:
    #     #轉df
        transform_df = pd.read_csv(csv_file,header=None)
first_cell = transform_df.iloc[0,0]  # 获取第0行数据
is_rent_time = first_cell == 'rent_time'  # 检查第0行是否等于'rent_time'
is_rent_time
if is_rent_time:
    transform_df.columns = transform_df.iloc[0]
    transform_df = transform_df.drop(0)

transform_df.to_csv(filename) ##要調整

In [37]:
transform_df

,rent_time,rent_station,return_time,return_station,rent,infodate
1,2022-09-30 21:00:00,捷運大湖公園站(2號出口),2022-09-30 21:00:00,安湖三號公園,00:12:27,2022-09-30
2,2022-09-30 21:00:00,捷運大湖公園站(2號出口),2022-09-30 21:00:00,康寧公園(北側),00:12:56,2022-09-30
3,2022-09-30 21:00:00,捷運大湖公園站(2號出口),2022-09-30 21:00:00,大湖國小,00:11:10,2022-09-30
4,2022-09-30 21:00:00,捷運大湖公園站(2號出口),2022-09-30 21:00:00,康樂綠地,00:29:07,2022-09-30
5,2022-09-30 22:00:00,捷運大湖公園站(2號出口),2022-09-30 22:00:00,大湖國小,00:03:55,2022-09-30
...,...,...,...,...,...,...
2632272,2022-09-23 14:00:00,成功路四段41巷口,2022-09-23 15:00:00,宏匯瑞光廣場(瑞光路),00:13:33,2022-09-23
2632273,2022-09-23 14:00:00,成功路四段41巷口,2022-09-23 15:00:00,星雲街47號,00:47:18,2022-09-23
2632274,2022-09-23 15:00:00,成功路四段41巷口,2022-09-23 15:00:00,康寧金湖路口,00:17:55,2022-09-23
2632275,2022-09-23 15:00:00,成功路四段41巷口,2022-09-23 16:00:00,南京東路六段368巷,00:18:43,2022-09-23


In [ ]:
#3-3 處理狀況二：將有錯誤的字符用,encoding_errors='replace' 處理
#把3-3 整合到step2內容
#df = pd.read_csv('202306_YouBike2.0▓╝├╥¿ΩÑd╕Ω«╞.csv',encoding_errors='replace')



with zipfile.ZipFile(zip_content, 'r') as zip_ref:
    data = zip_ref.namelist()
    for file in data:
        if '.csv' in file:
            file_in_zip =  file
    #解壓縮
    with zip_ref.open(file_in_zip) as csv_file:
    #     #轉df
        transform_df = pd.read_csv(csv_file,header=None,encoding_errors='replace')
first_cell = transform_df.iloc[0,0]  # 获取第0行数据
is_rent_time = first_cell == 'rent_time'  # 检查第0行是否等于'rent_time'
is_rent_time
if is_rent_time:
    transform_df.columns = transform_df.iloc[0]
    transform_df = transform_df.drop(0)

transform_df.to_csv(filename) ##要調整